### Problématique : Différencier les types d'utilisateur
Objectif : Déterminer, à partir des 48 premières heures, la classe d’un utilisateur qui n’a pas payé pendant ce laps de temps parmi les 3 suivantes :
* ne va jamais payer dans une période de 30 jours après les 48 premières heures
* va payer dans une période de 8 jours après les 48 premières heures et résilier dans les 30 jours après sa souscription
* va payer dans une période de 8 jours après les 48 premières heures et ne pas résilier son abonnement lors de son premier mois

### Import des librairies

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

### Visualisation des données

In [3]:
df = pd.read_parquet('app_ios_data_20220901_20230113.parquet', engine='pyarrow')
df.head()
print("Taille du dataframe : ", df.shape)

,amplitude_id,client_event_time,client_upload_time,country,device_family,device_model,event_id,created_at,event_type,language,...,push_notification_allowed,premium,no_of_custom_routines_saved,daily_reminder,days_completed_count,last_stretch_time,active_streak_count,longest_streak_count,apple_health,att
0,532864260650,2022-12-24 06:07:19.881,2022-12-24 06:07:19.906,United Kingdom,Apple iPhone,iPhone XR,1,2022-12-24 06:07:19.881,session_start,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
1,532864288194,2022-12-24 06:07:23.963,2022-12-24 06:07:23.986,Australia,Apple iPhone,iPhone 8,1,2022-12-24 06:07:23.963,session_start,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
2,532864260650,2022-12-24 06:07:24.875,2022-12-24 06:07:49.965,United Kingdom,Apple iPhone,iPhone XR,2,2022-12-24 06:07:24.875,onboarding_start,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
3,532864260650,2022-12-24 06:07:25.966,2022-12-24 06:07:49.965,United Kingdom,Apple iPhone,iPhone XR,4,2022-12-24 06:07:25.966,onboarding_page_view,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
4,532864260650,2022-12-24 06:07:26.192,2022-12-24 06:07:49.965,United Kingdom,Apple iPhone,iPhone XR,6,2022-12-24 06:07:26.192,first_app_open,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN


In [5]:
df["paying"] = df["paying"].replace(["nan", "true"], [0,1])

### Données des 48 premières heures

In [6]:
df48 = df[df["created_at"] < df["installed_at"] + pd.DateOffset(days=2)]
### On sélectionne seulement les données d'une durée de 48 heures après l'installation de l'application
df48.head()

,amplitude_id,client_event_time,client_upload_time,country,device_family,device_model,event_id,created_at,event_type,language,...,push_notification_allowed,premium,no_of_custom_routines_saved,daily_reminder,days_completed_count,last_stretch_time,active_streak_count,longest_streak_count,apple_health,att
0,532864260650,2022-12-24 06:07:19.881,2022-12-24 06:07:19.906,United Kingdom,Apple iPhone,iPhone XR,1,2022-12-24 06:07:19.881,session_start,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
1,532864288194,2022-12-24 06:07:23.963,2022-12-24 06:07:23.986,Australia,Apple iPhone,iPhone 8,1,2022-12-24 06:07:23.963,session_start,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
2,532864260650,2022-12-24 06:07:24.875,2022-12-24 06:07:49.965,United Kingdom,Apple iPhone,iPhone XR,2,2022-12-24 06:07:24.875,onboarding_start,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
3,532864260650,2022-12-24 06:07:25.966,2022-12-24 06:07:49.965,United Kingdom,Apple iPhone,iPhone XR,4,2022-12-24 06:07:25.966,onboarding_page_view,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
4,532864260650,2022-12-24 06:07:26.192,2022-12-24 06:07:49.965,United Kingdom,Apple iPhone,iPhone XR,6,2022-12-24 06:07:26.192,first_app_open,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN


### Suppression des payeurs dans les 48 premières heures

In [7]:
df48["paying"] = df["paying"].replace(["nan", "true"], [0,1])
paying48 = df48[["amplitude_id", "paying"]].groupby("amplitude_id").max()
### on obtient un dataframe qui, pour chaque utilisateur, nous indique s'il a payé durant les 48 premières heures
payers48_id = list(paying48[paying48["paying"] == 1].index)
### on récupère l'id des utilisateurs qui ont payé *payeurs*
non_payers_48 = df48[~(df48["amplitude_id"].isin(payers48_id))]
### on considère que les données des non payeurs dans les 48 premières heures

/var/folders/dd/0hw4ljv901vct22jhgypdgd00000gn/T/ipykernel_2254/2370419480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df48["paying"] = df["paying"].replace(["nan", "true"], [0,1])


### Suppression des utilisateurs ayant pour événements "payment_finish" ou "rc_trial_started_event"
Il y a également des utilisateurs qui ont les événements **payment_finish** et **rc_trial_started_event**. Ces utilisateurs là sont considérés comme des payeurs. Il faut donc les enlever.

In [9]:
payment_finish = non_payers_48[(non_payers_48["event_type"] == "payment_finish")]
### on récupère les lignes des utilisateurs qui ont l'événement payment_finish
rc_trial_started_event = non_payers_48[(non_payers_48["event_type"] == "rc_trial_started_event")]
### on récupère les lignes des utilisateurs qui ont l'événement rc_trial_started_event

payment_finish_id = list(payment_finish["amplitude_id"].unique())
rc_trial_started_event_id = list(rc_trial_started_event["amplitude_id"].unique())
### on récupères les ids des utilisateurs qui ont ces événements

id_to_remove = payment_finish_id + rc_trial_started_event_id
id_to_remove = set(id_to_remove)
### on en déduit les utilisateurs à supprimer 

In [10]:
non_payers_48 = non_payers_48[~(non_payers_48["amplitude_id"].isin(id_to_remove))]
### on obtient donc les données des utilisateurs non payeurs dans les 48 premières heures

### Création de la variable cible
On va maintenant récupérer les ids des utilisateurs des 3 catégories souhaitées.

#### va payer dans les 8 jours post 48 heures
On regarde d'abbord quels sont les utilisateurs qui paient dans une période de 8 jours suivant les 48h.

In [11]:
non_payers48_id = list(non_payers_48["amplitude_id"].unique()) 
### on récupère les ids des non payeurs durant les 48 premières heures

df10 = df[(df["client_event_time"] >= df["installed_at"] + pd.DateOffset(days=2))
         & (df["client_event_time"] < df["installed_at"] + pd.DateOffset(days=10))]

non_payers10 = df10[df10["amplitude_id"].isin(non_payers48_id)] 
### on récupère les événenements des non payeurs des 48 premières heures sur une période de 8 jours

In [13]:
subscription10_id_date = non_payers10[non_payers10["event_type"] == "rc_initial_purchase_event"][["amplitude_id", "created_at"]]
subscription10_id_date.rename(columns = {"created_at" : "subscription_date"}, inplace = True)
### on récupère les ids et la date de souscription des non payeurs des 48 premières heures
### qui souscrivent dans une période de 8 jours


#### va annuler dans les 30 jours suivant la souscription
Maintenant qu'on a récupéré les utilisateurs qui souscrivent dans une période de 8 jours après 48 premières heures, on regarde qui va résilier.

In [16]:
subscription10_id = list(subscription10_id_date["amplitude_id"].unique())
### on récupère les id des non payeurs des 48 premières heures qui souscrivent dans une période de 8 jours
non_payers30 = df[df["amplitude_id"].isin(subscription10_id)]
### on récupère les données des non payeurs des 48 premières heures qui souscrivent dans une période de 8 jours
non_payers30 = pd.merge(non_payers30, subscription10_id_date, on = "amplitude_id") 
### on ajoute la date de souscription à ces données
non_payers30 = non_payers30[(non_payers30["created_at"] >= non_payers30["subscription_date"]) 
                 & (non_payers30["created_at"] <= non_payers30["subscription_date"] + pd.DateOffset(days=30))]
### on ne garde seulement les données sur une période de 30 jours après la souscription d'un utilisateur

In [27]:
cancelers30_id = list(non_payers30[non_payers30["event_type"] == "rc_cancellation_event"]["amplitude_id"].unique())
### on en déduit la liste des non payeurs durant les 48 premières heures qui souscrivent un abonnement dans les 
### 8 jours suivant les 48 heures et qui résilient dans le mois suivant la date de souscription

#### ne va pas résilier dans les 30 jours après la souscription
On va faire la même chose qu'au dessus en regardant ceux qui n'ont pas résilié.

In [25]:
non_cancelers30_id = list(non_payers30[non_payers30["event_type"] != "rc_cancellation_event"]["amplitude_id"].unique())
### on en déduit la liste des non payeurs durant les 48 premières heures qui souscrivent un abonnement dans les 
### 8 jours suivant les 48 heures et qui ne résilient pas dans le mois suivant la date de souscription

#### ne va jamais payer
Pour finir, on va regarder quels sont les utilisateurs qui ne souscrivent jamais après dans une période de 30 jours après les 48 premières heures.

In [43]:
df30 = df[(df["client_event_time"] >= df["installed_at"] + pd.DateOffset(days=2))
         & (df["client_event_time"] < df["installed_at"] + pd.DateOffset(days=32))]
### on récupère les données sur une période de 30 jours après les 48 premières heures
non_payers30 = df30[df30["amplitude_id"].isin(non_payers48_id)]
### on observe ces données seulement pour les utilisateurs non payeurs durant les 48 premières heures
non_payers30["is_rc_initial_purchase_event"] = (non_payers30["event_type"] == "rc_initial_purchase_event").replace([True, False], [1,0])
### on créé une colonne qui indique si l'événement est une souscription
subscription = non_payers30[["amplitude_id", "is_rc_initial_purchase_event"]].groupby("amplitude_id").max()
### on obtient un dataframe qui, pour chaque utilisateur, nous indique s'il a souscrit dans la période de 30 jours

/var/folders/dd/0hw4ljv901vct22jhgypdgd00000gn/T/ipykernel_2254/3209385468.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_payers30["is_rc_initial_purchase_event"] = (non_payers30["event_type"] == "rc_initial_purchase_event").replace([True, False], [1,0])


In [4]:
never_subscribed_id = list(subscription[subscription["is_rc_initial_purchase_event"] == 0]["amplitude_id"].unique())
### liste des ids des utilisateurs qui n'ont jamais souscrit

NameError: name 'subscription' is not defined